In [1]:
# Import dependencies
import sys
sys.path.append('./Resources')
import pymongo
import requests
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2022)


   # Data pull and pre-processing

In [2]:
# Run Census Search to retrieve data on all county codes ( ACS5 Census)

def get_census(year):
    c = Census(api_key, year=year)
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), 
                         {'for': 'county:*'})
                       

# Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

# Column Reordering
    census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "county": "County"
                                      })

# Add in Poverty Rate (Poverty Count / Population)
    census_pd["Poverty Rate"] = 100 * \
        census_pd["Poverty Count"].astype(
            int) / census_pd["Population"].astype(int)

# Final DataFrame
    census_pd = census_pd[["Name","County", "Population", "Median Age", "Household Income",
                           "Per Capita Income", "Poverty Count", "Poverty Rate"]]

    census_pd['county_name'] = census_pd['Name'].str.replace(r" County,(.*)",'').str.upper()
    census_pd['state'] = census_pd['Name'].str.replace(r"(.*) County, ",'').str.upper()

    
    return census_pd

census_2012 = get_census(2012)
census_2012.head()
census_2012['year'] = 2012

census_2016 = get_census(2016)
census_2016.head()
census_2016['year'] = 2016

census_2020 = get_census(2020)
census_2020['year'] = 2020
census_2020.head()

census = pd.concat([census_2012,census_2016,census_2020])


C:\Users\nico.cosereanu\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
C:\Users\nico.cosereanu\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: The default value of regex will change from True to False in a future version.


In [3]:
census.dtypes

Name                  object
County                object
Population           float64
Median Age           float64
Household Income     float64
Per Capita Income    float64
Poverty Count        float64
Poverty Rate         float64
county_name           object
state                 object
year                   int64
dtype: object

In [4]:
# Perform some data cleansing options Drop records with Household Income less then 0
census = census[census['Household Income']>0]
census

,Name,County,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,county_name,state,year
0,"Linn County, Missouri",115,12668.0,43.0,39028.0,20968.0,1678.0,13.245974,LINN,MISSOURI,2012
1,"Howell County, Missouri",091,40330.0,39.6,34148.0,17763.0,8187.0,20.300025,HOWELL,MISSOURI,2012
2,"Johnson County, Missouri",101,52964.0,29.7,47960.0,21375.0,8075.0,15.246205,JOHNSON,MISSOURI,2012
3,"Laclede County, Missouri",105,35507.0,39.1,39101.0,19788.0,6478.0,18.244290,LACLEDE,MISSOURI,2012
4,"Maries County, Missouri",125,9140.0,43.1,44885.0,21883.0,1286.0,14.070022,MARIES,MISSOURI,2012
...,...,...,...,...,...,...,...,...,...,...,...
3216,"Renville County, Minnesota",129,14572.0,44.0,58542.0,31243.0,1373.0,9.422180,RENVILLE,MINNESOTA,2020
3217,"Roseau County, Minnesota",135,15259.0,41.6,62304.0,31452.0,1133.0,7.425126,ROSEAU,MINNESOTA,2020
3218,"Sherburne County, Minnesota",141,96015.0,36.1,88671.0,36022.0,4953.0,5.158569,SHERBURNE,MINNESOTA,2020
3219,"Steele County, Minnesota",147,36710.0,39.2,68172.0,34648.0,2887.0,7.864342,STEELE,MINNESOTA,2020


In [5]:
# Import the winner values by county
winners_pd=pd.read_csv("Resources/countypres_2000-2020_with_winner.csv")
winners_pd

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,Concat,Winner
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL,2000ALABAMAAUTAUGA,Republican
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL,2000ALABAMAAUTAUGA,Republican
2,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL,2000ALABAMAAUTAUGA,Republican
3,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL,2000ALABAMAAUTAUGA,Republican
4,2000,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL,2000ALABAMABALDWIN,Republican
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72612,2020,WYOMING,WY,WASHAKIE,56043.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,3245,4032,20220315,TOTAL,2020WYOMINGWASHAKIE,Republican
72613,2020,WYOMING,WY,WESTON,56045.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,360,3560,20220315,TOTAL,2020WYOMINGWESTON,Republican
72614,2020,WYOMING,WY,WESTON,56045.0,US PRESIDENT,JO JORGENSEN,LIBERTARIAN,46,3560,20220315,TOTAL,2020WYOMINGWESTON,Republican
72615,2020,WYOMING,WY,WESTON,56045.0,US PRESIDENT,OTHER,OTHER,47,3560,20220315,TOTAL,2020WYOMINGWESTON,Republican


In [6]:
winners_pd = winners_pd[((winners_pd['year'] == 2012) | (winners_pd['year'] == 2016) | (winners_pd['year'] == 2020))]
winners_pd = winners_pd[((winners_pd['party'] == 'DEMOCRAT') | (winners_pd['party'] == 'REPUBLICAN'))]
winners_pd=winners_pd.drop_duplicates(subset=['state_po','county_name', 'year'],keep='first')

In [7]:
winners_pd

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,Concat,Winner
31166,2012,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,BARACK OBAMA,DEMOCRAT,6363,23932,20220315,TOTAL,2012ALABAMAAUTAUGA,Republican
31169,2012,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,BARACK OBAMA,DEMOCRAT,18424,85338,20220315,TOTAL,2012ALABAMABALDWIN,Republican
31172,2012,ALABAMA,AL,BARBOUR,1005.0,US PRESIDENT,BARACK OBAMA,DEMOCRAT,5912,11509,20220315,TOTAL,2012ALABAMABARBOUR,Democrat
31175,2012,ALABAMA,AL,BIBB,1007.0,US PRESIDENT,BARACK OBAMA,DEMOCRAT,2202,8420,20220315,TOTAL,2012ALABAMABIBB,Republican
31178,2012,ALABAMA,AL,BLOUNT,1009.0,US PRESIDENT,BARACK OBAMA,DEMOCRAT,2970,24006,20220315,TOTAL,2012ALABAMABLOUNT,Republican
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72597,2020,WYOMING,WY,SWEETWATER,56037.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,3823,16698,20220315,TOTAL,2020WYOMINGSWEETWATER,Republican
72601,2020,WYOMING,WY,TETON,56039.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,9848,14787,20220315,TOTAL,2020WYOMINGTETON,Democrat
72605,2020,WYOMING,WY,UINTA,56041.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,1591,9459,20220315,TOTAL,2020WYOMINGUINTA,Republican
72609,2020,WYOMING,WY,WASHAKIE,56043.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,651,4032,20220315,TOTAL,2020WYOMINGWASHAKIE,Republican


In [8]:
# Merge the winner info with the previosuly established DF
cleaned_census_data = pd.merge(census, winners_pd, how='left', left_on=['year','state', 'county_name'], right_on=['year','state', 'county_name'])

cleaned_census_data = cleaned_census_data[['year','state','state_po','county_name','Population','Median Age', 'Household Income', 'Per Capita Income', 'Poverty Rate', 'Winner']]
cleaned_census_data

,year,state,state_po,county_name,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Winner
0,2012,MISSOURI,MO,LINN,12668.0,43.0,39028.0,20968.0,13.245974,Republican
1,2012,MISSOURI,MO,HOWELL,40330.0,39.6,34148.0,17763.0,20.300025,Republican
2,2012,MISSOURI,MO,JOHNSON,52964.0,29.7,47960.0,21375.0,15.246205,Republican
3,2012,MISSOURI,MO,LACLEDE,35507.0,39.1,39101.0,19788.0,18.244290,Republican
4,2012,MISSOURI,MO,MARIES,9140.0,43.1,44885.0,21883.0,14.070022,Republican
...,...,...,...,...,...,...,...,...,...,...
9656,2020,MINNESOTA,MN,RENVILLE,14572.0,44.0,58542.0,31243.0,9.422180,Republican
9657,2020,MINNESOTA,MN,ROSEAU,15259.0,41.6,62304.0,31452.0,7.425126,Republican
9658,2020,MINNESOTA,MN,SHERBURNE,96015.0,36.1,88671.0,36022.0,5.158569,Republican
9659,2020,MINNESOTA,MN,STEELE,36710.0,39.2,68172.0,34648.0,7.864342,Republican


In [9]:
cleaned_census_data.dropna(subset =["state_po"],inplace=True)

In [10]:
# Create the final dataset for ML application
db_data = cleaned_census_data
db_data

,year,state,state_po,county_name,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Winner
0,2012,MISSOURI,MO,LINN,12668.0,43.0,39028.0,20968.0,13.245974,Republican
1,2012,MISSOURI,MO,HOWELL,40330.0,39.6,34148.0,17763.0,20.300025,Republican
2,2012,MISSOURI,MO,JOHNSON,52964.0,29.7,47960.0,21375.0,15.246205,Republican
3,2012,MISSOURI,MO,LACLEDE,35507.0,39.1,39101.0,19788.0,18.244290,Republican
4,2012,MISSOURI,MO,MARIES,9140.0,43.1,44885.0,21883.0,14.070022,Republican
...,...,...,...,...,...,...,...,...,...,...
9656,2020,MINNESOTA,MN,RENVILLE,14572.0,44.0,58542.0,31243.0,9.422180,Republican
9657,2020,MINNESOTA,MN,ROSEAU,15259.0,41.6,62304.0,31452.0,7.425126,Republican
9658,2020,MINNESOTA,MN,SHERBURNE,96015.0,36.1,88671.0,36022.0,5.158569,Republican
9659,2020,MINNESOTA,MN,STEELE,36710.0,39.2,68172.0,34648.0,7.864342,Republican


In [11]:
#Uploading The Pandas DataFrame to MongoDB
# Convert dataframe to dictinary
db_data_ML=db_data.to_dict("records")
db_data_ML

[{'year': 2012,
  'state': 'MISSOURI',
  'state_po': 'MO',
  'county_name': 'LINN',
  'Population': 12668.0,
  'Median Age': 43.0,
  'Household Income': 39028.0,
  'Per Capita Income': 20968.0,
  'Poverty Rate': 13.245974107988634,
  'Winner': 'Republican'},
 {'year': 2012,
  'state': 'MISSOURI',
  'state_po': 'MO',
  'county_name': 'HOWELL',
  'Population': 40330.0,
  'Median Age': 39.6,
  'Household Income': 34148.0,
  'Per Capita Income': 17763.0,
  'Poverty Rate': 20.30002479543764,
  'Winner': 'Republican'},
 {'year': 2012,
  'state': 'MISSOURI',
  'state_po': 'MO',
  'county_name': 'JOHNSON',
  'Population': 52964.0,
  'Median Age': 29.7,
  'Household Income': 47960.0,
  'Per Capita Income': 21375.0,
  'Poverty Rate': 15.246204969413187,
  'Winner': 'Republican'},
 {'year': 2012,
  'state': 'MISSOURI',
  'state_po': 'MO',
  'county_name': 'LACLEDE',
  'Population': 35507.0,
  'Median Age': 39.1,
  'Household Income': 39101.0,
  'Per Capita Income': 19788.0,
  'Poverty Rate': 18.2

In [12]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [13]:
# Export data to MongoDB
db = client.census_DB
db.census.drop()
collection = db['census']
collection.insert_many(db_data_ML)

In [14]:
results = client.census_DB.census.find()
cluster_data = []
for result in results:
    del result['_id']
    cluster_data.append(result)

In [15]:
cluster_data = pd.DataFrame(cluster_data)
cluster_data

,year,state,state_po,county_name,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Winner
0,2012,MISSOURI,MO,LINN,12668.0,43.0,39028.0,20968.0,13.245974,Republican
1,2012,MISSOURI,MO,HOWELL,40330.0,39.6,34148.0,17763.0,20.300025,Republican
2,2012,MISSOURI,MO,JOHNSON,52964.0,29.7,47960.0,21375.0,15.246205,Republican
3,2012,MISSOURI,MO,LACLEDE,35507.0,39.1,39101.0,19788.0,18.244290,Republican
4,2012,MISSOURI,MO,MARIES,9140.0,43.1,44885.0,21883.0,14.070022,Republican
...,...,...,...,...,...,...,...,...,...,...
9002,2020,MINNESOTA,MN,RENVILLE,14572.0,44.0,58542.0,31243.0,9.422180,Republican
9003,2020,MINNESOTA,MN,ROSEAU,15259.0,41.6,62304.0,31452.0,7.425126,Republican
9004,2020,MINNESOTA,MN,SHERBURNE,96015.0,36.1,88671.0,36022.0,5.158569,Republican
9005,2020,MINNESOTA,MN,STEELE,36710.0,39.2,68172.0,34648.0,7.864342,Republican
